In [21]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statsmodels.api as sm
from scipy.special import gamma, loggamma, factorial
import scipy.stats
from scipy.interpolate import interp1d
from matplotlib.ticker import AutoMinorLocator 
from matplotlib import rc, font_manager
from matplotlib.lines import Line2D
from matplotlib import colors as mcolors
from matplotlib import legend_handler
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
import collections
import shapefile
import matplotlib
from OSGridConverter import latlong2grid
import shapely.geometry
import shapely.ops 
from descartes.patch import PolygonPatch
from geopy.geocoders import Nominatim
import geopandas as gpd
import requests
import io
import pysal
from pysal.lib import weights
import warnings
warnings.filterwarnings('ignore')

In [22]:
def counts(gdf_collisions, gdf_geography):
    
    gdf_geography['index'] = [i for i in range(len(gdf_geography))]
    #Add geography to each event, aggregate events with same geography, add number of counts to each geography
    gdf_collisions['Counts'] = ['Counts' for i in range(len(gdf_collisions))]
    gdf_join = gpd.sjoin(gdf_geography, gdf_collisions)
    dfpivot = pd.pivot_table(gdf_join, index='index', columns='Counts', aggfunc={'Counts':len})
    dfpivot.columns = dfpivot.columns.droplevel()
    gdf_counts = gdf_geography.merge(dfpivot, how='left', on='index')
    gdf_counts['Counts'] = gdf_counts['Counts'].fillna(0)
    gdf_counts['Events/km_road'] = np.zeros(len(gdf_counts))
    for j in range(len(gdf_counts)):
        if float(gdf_counts.loc[j, 'length_roa']) != 0:
            gdf_counts.loc[j, 'Events/km_road'] = gdf_counts.loc[j, 'Counts']/float(gdf_counts.loc[j, 'length_roa'])*1e3
    return gdf_counts


In [23]:
def spatial_lag(gdf, w_neighbours):

    WY = []
    for i in range(len(gdf)):
        wy = 0
        index_neighbours = w_neighbours[i]
        for j in index_neighbours:
            wy += gdf.loc[j, 'Events/km_road']
        wy /= len(index_neighbours)
        WY.append(wy)
    gdf['WY'] = WY
    return gdf

In [24]:
def find_cumulatives(gdf):    
    
    # Find cumulative distribution for WY
    gdf = gdf.sort_values(by='WY').reset_index(drop=True)
    gdf['F_WY'] = np.arange(len(gdf))/len(gdf)
    
    # Find cumulative distribution for Y
    gdf = gdf.sort_values(by='Events/km_road').reset_index(drop=True)
    gdf['F_Y'] = np.arange(len(gdf))/len(gdf)
    return gdf

In [25]:
def gini_gamma(gdf):
    
    Y = gdf['Events/km_road']
    F_Y = list(gdf['F_Y'])
    F_WY = list(gdf['F_WY'])
    cov_Y = 0
    cov_WY = 0
    mean_Y = np.mean(Y)
    mean_F_Y = np.mean(F_Y)
    mean_F_WY = np.mean(F_WY)
    index_non_zero = 0
    while Y[index_non_zero]==0:
        index_non_zero+=1
    for i in range(len(Y)):
        cov_Y += (Y[i]-mean_Y)*(F_Y[i]-mean_F_Y)
        cov_WY += (Y[i]-mean_Y)*(F_WY[i]-mean_F_WY)
    cov_Y = cov_Y/len(Y)
    cov_WY = cov_WY/len(Y)
    # Gini
    gini = 2*cov_Y/mean_Y
    # spatial Gini
    gini_s = 2*cov_WY/mean_Y
    # Gini correlation
    gamma = gini_s/gini
    return gini, gini_s, gamma

In [26]:
def moransI(gdf, w_neighbours):
    
    Y = gdf['Events/km_road']
    mean_Y = np.mean(Y)
    numerator = 0
    denominator = 0
    W = 0
    for i in range(len(Y)):
        index_neighbours = w_neighbours[i]
        denominator += (Y[i] - mean_Y)**2
        W += len(index_neighbours)
        for j in index_neighbours:
            numerator += (Y[i]-mean_Y)*(Y[j]-mean_Y)
    I = (len(Y)/W)*(numerator/denominator)
    return I

In [27]:
url = 'https://raw.githubusercontent.com/CrmnCA/concentration-collisions/master/'
years = [15,16,17,18,19]
download = requests.get(url + 'Accidents' + str(years[0]) + '.csv').content
df = pd.read_csv(io.StringIO(download.decode('utf-8')), low_memory=False)
index_to_drop = []
for i in range(len(df)):
    if type(df.loc[i,'LSOA_of_Accident_Location']) != str:
        index_to_drop.append(i)
df = df.drop(index_to_drop).reset_index(drop=True)
df = df[df['LSOA_of_Accident_Location'] < 'F'] #Select collisions in England
print('Total events England in year 20' + str(years[0]) +': ', len(df))
df = df[df['Accident_Severity']<3].reset_index(drop=True) #Select fatal and serious collisions (1:fatal, 2:serious, 3:minor)
print('Filtered events England in year 20' + str(years[0]) +': ', len(df))
df = df[['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Date', 'Time']]
for i in range(1, len(years)):
    download = requests.get(url + 'Accidents' + str(years[i]) + '.csv').content
    df_year = pd.read_csv(io.StringIO(download.decode('utf-8')), low_memory=False)
    index_to_drop = []
    for j in range(len(df_year)):
        if type(df_year.loc[j,'LSOA_of_Accident_Location']) != str:
            index_to_drop.append(j)
    df_year = df_year.drop(index_to_drop).reset_index(drop=True)
    df_year = df_year[df_year['LSOA_of_Accident_Location'] < 'F'] #Select collisions in England
    print('Total events England in year 20' + str(years[i]) +': ', len(df_year))
    df_year = df_year[df_year['Accident_Severity']<3].reset_index(drop=True) #Select fatal and serious collisions (1:fatal, 2:serious, 3:minor)
    print('Filtered events England in year 20' + str(years[i]) +': ', len(df_year))
    df_year = df_year[['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Date', 'Time']]
    nan_value = float('NaN')
    df_year.replace('', nan_value)
    df_year = df_year.dropna().reset_index(drop=True)
    df = pd.concat([df, df_year], ignore_index=True)
gdf_collisions = gpd.GeoDataFrame(df, crs = 'EPSG:27700', geometry=gpd.points_from_xy(df.Location_Easting_OSGR, df.Location_Northing_OSGR))
gdf_collisions = gdf_collisions[['geometry', 'Date', 'Time']]


Total events England in year 2015:  125637
Filtered events England in year 2015:  19006
Total events England in year 2016:  123348
Filtered events England in year 2016:  20840
Total events England in year 2017:  118297
Filtered events England in year 2017:  21758
Total events England in year 2018:  111978
Filtered events England in year 2018:  22323
Total events England in year 2019:  113222
Filtered events England in year 2019:  23586


In [28]:
url = 'https://raw.githubusercontent.com/CrmnCA/concentration-collisions/master/Hexagons_cities_with_roadway_length.geojson'
gdf_hexagons_cities=gpd.read_file(url, crs='EPSG:27700')
ginis, ginis_s, gammas, morans = [], [], [], []
for i in range(0,32):
    gdf_hexagons_city = gdf_hexagons_cities[gdf_hexagons_cities['city']==i].reset_index(drop=True)
    gdf_count = counts(gdf_collisions, gdf_hexagons_city)  
    w_neighbours = weights.fuzzy_contiguity(gdf_count, tolerance=0.001, buffering=True)
    w_neighbours = {i:list(w_neighbours[i].keys()) for i in range(len(gdf_count))} 
    moran = moransI(gdf_count, w_neighbours)
    morans.append(moran)
    gdf_count = spatial_lag(gdf_count, w_neighbours)
    gdf_count = find_cumulatives(gdf_count)
    gini, gini_s, gamma = gini_gamma(gdf_count)
    ginis.append(gini)
    ginis_s.append(gini_s)
    gammas.append(gamma)   
    
print(pd.DataFrame({'Gini index':ginis, 'Morans I': morans, 'Gini correlation': gammas}))



    Gini index  Morans I  Gini correlation
0     0.596351  0.547753          0.611252
1     0.625345  0.227980          0.365338
2     0.556988  0.392787          0.553605
3     0.612705  0.275978          0.371154
4     0.563777  0.341889          0.558914
5     0.556300  0.276350          0.391712
6     0.565544  0.259343          0.319912
7     0.631272  0.325102          0.461504
8     0.565721  0.332524          0.489079
9     0.669120  0.353768          0.516559
10    0.627225  0.467952          0.604073
11    0.551557  0.348610          0.570412
12    0.607338  0.187227          0.413179
13    0.547669  0.255661          0.372395
14    0.642275  0.161287          0.317746
15    0.665074  0.155498          0.325759
16    0.607262  0.108229          0.289423
17    0.699899  0.249570          0.456032
18    0.569642  0.251412          0.424745
19    0.612479  0.218105          0.341946
20    0.569052  0.320517          0.543589
21    0.588833  0.119748          0.460038
22    0.610